In [14]:
import random
import numpy as np
import gurobipy as gp
from gurobipy import GRB
import matplotlib.pyplot as plt

# Variables
num_rows = 10  # Number of rows
num_cols = 10 # Number of columns
K = 4   # Number of robots
T = 40  # Time
depots = 4
max_coverage_per_robot = 5
initial_fuel = [2 * num_rows * num_cols]
min_fuel_threshold = 1.0
fuel_capacity = [np.sqrt(8) * 2] * K  # Fuel capacity for each robot

# Create a 2D array for targets
n = np.zeros((num_rows, num_cols), dtype=int)

# Initialize targets randomly (you can replace this with your target initialization logic)
for i in range(num_rows):
    for j in range(num_cols):
        n[i, j] = random.randint(0, 1)

fuel_consumption = [[[0.1] * T for _ in range(num_cols)] for _ in range(num_rows)]  # Max amount of fuel per robot

# Specify different initial locations for each robot at corners
initial_locations = [
    (0, 0),             # Top-left corner
    (0, num_cols-1),    # Top-right corner
    (num_rows-1, 0),    # Bottom-left corner
    (num_rows-1, num_cols-1)  # Bottom-right corner
]
initial_location = [col + row * num_cols for row, col in initial_locations]
coverage_cost = [1.0]  * (num_rows * num_cols)  # Coverage cost for each target

model = gp.Model("MRPCP")   # Create Gurobi model

# Create binary variables for robot locations and target coverage
x = model.addVars(K, num_rows, num_cols, T, vtype=GRB.BINARY, name="x")
target_covered = model.addVars(K, num_rows, num_cols, T, vtype=GRB.BINARY, name="target_covered")

# Create binary variables for charging depots
charging_depot = model.addVars(num_rows, num_cols, vtype=GRB.BINARY, name="charging_depot")

# Create variables for fuel levels
fuel = model.addVars(K, T, name="fuel")
initial_fuel = model.addVars(K, name="initial_fuel")

# --------Initial conditions------------
for i in range(K):
    model.addConstr(initial_fuel[i] == 100)
    model.addConstr(x[i, initial_location[i] // num_cols, initial_location[i] % num_cols, 0] == 1)
    model.addConstr(fuel[i, 0] == initial_fuel[i])  # Fix indexing

# Define the coverage cost (minimize)
model.setObjective(gp.quicksum(coverage_cost[j] * target_covered[i, j // num_cols, j % num_cols, t]
                               for j in range(num_rows * num_cols) for i in range(K) for t in range(T)),
                   GRB.MINIMIZE)

# -------- B. Degree Constraints (6), (7), (8), (9), (10) ------------

# Each target must be covered at least once
for j in range(num_rows):
    for k in range(num_cols):
        model.addConstr(gp.quicksum(target_covered[i, j, k, t] for i in range(K) for t in range(T)) >= 1)

# (6) and (7) Only one robot arrives to and leaves from a target
for j in range(num_rows):
    for k in range(num_cols):
        model.addConstr(gp.quicksum(target_covered[i, j, k, t] for i in range(K) for t in range(T)) <= 1)
        
# (8) and (9) Begin and end at same position - Ensure each robot starts and ends at the same position (corner)
for i in range(K):
    start_row, start_col = initial_locations[i]
    model.addConstr(gp.quicksum(x[i, start_row, start_col, t] for t in range(T)) == 1)  # Ensure it starts at the corner
    model.addConstr(gp.quicksum(x[i, start_row, start_col, t] for t in range(T)) == 1)  # Ensure it ends at the corner

# (10) Ensure that every robot that enters a target leaves the target
for j in range(num_rows):
    for k in range(num_cols):
        model.addConstr(gp.quicksum(target_covered[i, j, k, t] for i in range(K) for t in range(T // 2)) == gp.quicksum(target_covered[i, j, k, t] for i in range(K) for t in range(T // 2, T)))

# ------------- C. Capacity & Flow Constraints (11), (12), (13), (14) -------------------

# (11) and (12) flow constraints
for i in range(K):
    for j in range(num_rows):
        for k in range(num_cols):
            for t in range(T - 1):
                model.addConstr(x[i, j, k, t + 1] - x[i, j, k, t] == target_covered[i, j, k, t] - target_covered[i, j, k, t + 1])

# (13) Make sure target capacity doesn't change when passing through a depot
for i in range(K):
    for j in range(num_rows):
        for k in range(num_cols):
            for t in range(T):
                if t > 0:
                    model.addConstr(target_covered[i, j, k, t] <= target_covered[i, j, k, t - 1] + charging_depot[j, k])

# (14) Ensure that target capacity for each robot doesn't exceed |T|
for i in range(K):
    for j in range(num_rows):
        for k in range(num_cols):
            model.addConstr(gp.quicksum(target_covered[i, j, k, t] for t in range(T)) <= 1)


# ------------- D. Fuel Constraints (15), (16), (17), (18), (19), (20) -------------------

# (15) and (16) model the fuel consumption when a robot moves from one target to another. 
for i in range(K):
    for j in range(num_rows):
        for k in range(num_cols):
            for t in range(T - 1):
                # If robot i moves from target (j, k) to another target at time t
                model.addConstr(fuel[i, t + 1] == fuel[i, t] - gp.quicksum(fuel_consumption[j][k][t] * x[i, j, k, t] for j in range(num_rows) for k in range(num_cols)))

# (17) and (18) model the fuel consumption when a robot moves from a depot to a target
for i in range(K):
    for j in range(num_rows):
        for k in range(num_cols):
            for t in range(1, T):
                # If robot i enters target (j, k) at time t
                model.addConstr(fuel[i, t] == fuel_capacity[i] - gp.quicksum(fuel_consumption[j][k][t] * x[i, j, k, t] for j in range(num_rows) for k in range(num_cols)))
            
# (19) models the fuel consumption when a robot moves from a target to a depot.
for i in range(K):
    for t in range(T):
        model.addConstr(fuel[i, t] <= fuel_capacity[i] - gp.quicksum(fuel_consumption[j][k][t] * x[i, j, k, t] for j in range(num_rows) for k in range(num_cols)))
            
# Optimize the model
model.optimize()

if model.status == GRB.OPTIMAL:
    print("Optimal solution found. Total cost:", model.ObjVal)

    # Extract the variables and values
    x_values = model.getAttr("X", x)
    target_covered_values = model.getAttr("X", target_covered)
    charging_depot_values = model.getAttr("X", charging_depot)

    # Extract robot paths
    robot_paths = [[] for _ in range(K)]
    for i in range(K):
        for j in range(num_rows):
            for k in range(num_cols):
                for t in range(T):
                    if x_values[i, j, k, t] > 0.5:
                        robot_paths[i].append((j, k, t))  # Store (row, col, time) tuples

    # Print the paths for each robot in the console
    # for i in range(K):
    #     print(f"Robot {i + 1} Path:")
    #     path = robot_paths[i]
    #     for step in path:
    #         print(f"Time {step[2]}: Row {step[0]}, Column {step[1]}")
    #     print("\n")

    # Plot the paths of the robots and targets in the 2D array
    plt.figure(figsize=(8, 8))
    
    # Define colors for each robot
    colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k']
    
    # Plot all targets as white dots
    for j in range(num_rows):
        for k in range(num_cols):
            plt.scatter(k, j, color='k', s=100, marker='X')
    
    for i in range(K):
        path = robot_paths[i]
        color = colors[i % len(colors)]  # Cycle through colors if more than 7 robots
    
        # Plot the start point
        start_row, start_col, _ = path[0]
        plt.scatter(start_col, start_row, color=color, s=50, label=f'Robot {i + 1} Start')
    
        for m in range(len(path) - 1):
            current_row, current_col, current_time = path[m]
            next_row, next_col, next_time = path[m + 1]
    
            # Plot the current node
            plt.scatter(current_col, current_row, color=color, s=30)
    
            plt.plot([current_col, next_col], [current_row, next_row], color=color, linestyle='-', linewidth=2)
    
        # Plot the end point
        end_row, end_col, _ = path[-1]
        plt.scatter(end_col, end_row, color=color, s=50, label=f'Robot {i + 1} End')
    
    plt.title('Paths of Robots and Targets in 2D Array')
    plt.xlabel('Columns')
    plt.ylabel('Rows')
    plt.legend()
    plt.grid(True)
    plt.show()
else:
    print("No solution found. Status code:", model.status)

model.dispose();   

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (linux64)

CPU model: Intel(R) Core(TM) i9-10900K CPU @ 3.70GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 20 logical processors, using up to 20 threads

Academic license - for non-commercial use only - registered to sdholmes@wpi.edu
Optimize a model with 63280 rows, 32264 columns and 3356496 nonzeros
Model fingerprint: 0x7a14bda2
Variable types: 164 continuous, 32100 integer (32100 binary)
Coefficient statistics:
  Matrix range     [1e-01, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Presolve removed 15600 rows and 100 columns
Presolve time: 0.42s

Explored 0 nodes (0 simplex iterations) in 0.73 seconds (0.52 work units)
Thread count was 1 (of 20 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -
No solution found. Status code: 3
